In [ ]:
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np

# -----------------------------
#  CONFIG
# -----------------------------
ANNOTATION_FILE = "annotations.xml"    # path to XML file
IMAGE_FOLDER = "images"                # folder that has scene00001.png, scene00006.png, etc.
OUTPUT_FOLDER = "annotated_output"     # saves annotated images

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# -----------------------------
#  PARSE XML
# -----------------------------
tree = ET.parse(ANNOTATION_FILE)
root = tree.getroot()

# CVAT 1.1 stores images under <image> tags
images = root.findall("image")

print(f"Found {len(images)} frames in XML.")

# -----------------------------
#  FUNCTION: Convert polyline string to list of points
# -----------------------------
def parse_points(points_string):
    pts = []
    for p in points_string.split(";"):
        if p.strip():
            x, y = map(float, p.split(","))
            pts.append([int(x), int(y)])
    return np.array(pts, dtype=np.int32)

# -----------------------------
#  PROCESS EACH IMAGE
# -----------------------------
for img_node in images:
    img_name = img_node.attrib["name"]
    img_path = os.path.join(IMAGE_FOLDER, img_name)

    if not os.path.exists(img_path):
        print(f"[WARNING] Image not found: {img_path}")
        continue

    image = cv2.imread(img_path)

    # get all polylines
    for poly in img_node.findall("polyline"):
        points_str = poly.attrib["points"]
        pts = parse_points(points_str)

        # Draw the polyline
        cv2.polylines(image, [pts], isClosed=False, color=(0, 255, 0), thickness=2)

    # Save output
    out_path = os.path.join(OUTPUT_FOLDER, img_name)
    cv2.imwrite(out_path, image)
    print(f"[OK] Saved -> {out_path}")

print("\nDONE. Annotated images saved in:", OUTPUT_FOLDER)